<font color='tomato'><font color="#CC3D3D"><p>
# Cornac
https://cornac.preferred.ai/
    
<img align='left' src='http://drive.google.com/uc?export=view&id=1dFW4nXMa6HKnUldt1M7wji5f0_wFidYV' /><br>    

- **Cornac**은 Multimodal 추천시스템을 위한 비교 프레임워크(comparative framework)
- Side information(예: 항목 설명 텍스트 및 이미지, 소셜 네트워크 등)를 활용하는 모델을 편리하게 작업할 수 있도록 하는 데 중점
- 새로운 모델을 빠르게 실험하고 간단하게 구현이 가능
- 기존 머신 러닝 라이브러리(예: TensorFlow, PyTorch)와 호환성 높음
- 추천 알고리즘의 평가 및 재현성을 위해 ACM RecSys 2023에서 권장하는 프레임워크 중 하나
- Cornac의 실험절차    
<img src=https://github.com/PreferredAI/cornac/raw/master/flow.jpg>

### Setup

In [14]:
# !pip install cornac==1.17
# !pip install dgl

In [15]:
import pandas as pd
import numpy as np

# MS recommenders API 
import sys
sys.path.append('/msr')  # 본인이 msr.zip 압축을 푼 위치를 확인(셀에서 pwd 명령어 실행) 후 변경해야 함. 
                                                # 윈도우에서는 폴더 구분자를 // 또는 \\로 해야 함.  
from msr.cornac_utils import predict_ranking
from msr.python_splitters import python_stratified_split

# Cornac API 
import cornac
print(f"Cornac version: {cornac.__version__}")
from cornac.eval_methods import BaseMethod, RatioSplit, StratifiedSplit, CrossValidation #평가방법
from cornac.models import BiVAECF, LightGCN, EASE, NGCF, WMF
from cornac.metrics import Precision, Recall, NDCG, AUC, MAP #평가 척도
#from cornac.hyperopt import Discrete, Continuous
#from cornac.hyperopt import GridSearch, RandomSearch

Cornac version: 1.17


In [16]:
# Data column definition
DEFAULT_USER_COL = 'resume_seq'
DEFAULT_ITEM_COL = 'recruitment_seq'
DEFAULT_RATING_COL = 'rating' 
DEFAULT_PREDICTION_COL = 'prediction'

# Top k items to recommend
TOP_K = 15

# Random seed, Verbose, etc.
SEED = 202311
VERBOSE = True

### Data Preparation

In [17]:
# 데이터 로딩
data = pd.read_csv('data/apply_train.csv')
data[DEFAULT_RATING_COL] = 1  # Cornac에서 지정한 데이터형식(UIR: User, Item, Rating)에 따라

In [18]:
# 데이터 분할
train, test = python_stratified_split(
    data, 
    filter_by="user", 
    ratio=0.7, #비율을 높이면 테스트유저가 팍줄어버린다
    col_user=DEFAULT_USER_COL, col_item=DEFAULT_ITEM_COL,
    seed=SEED
)

print(
    "ratings per train user: ", train.groupby(DEFAULT_USER_COL).size().mean(), 
    "\nratings per test user: ", test.groupby(DEFAULT_USER_COL).size().mean()
) 

#train, test = [], []
#df_groupby = data.groupby(DEFAULT_USER_COL)[DEFAULT_ITEM_COL].apply(list)
#for uid, iids in zip(df_groupby.index.tolist(), df_groupby.values.tolist()):
#    for iid in iids[:-1]:
#        train.append([uid,iid])
#    test.append([uid, iids[-1]])    
#train = pd.DataFrame(train); train.columns = {DEFAULT_USER_COL, DEFAULT_ITEM_COL}
#test = pd.DataFrame(test); test.columns = {DEFAULT_USER_COL, DEFAULT_ITEM_COL}
#train[DEFAULT_RATING_COL] = 1.0
#test[DEFAULT_RATING_COL] = 1.0
# 사용자가 지원한 곳중 맨 마지막만 빼서 테스트 데이터로 만든것

ratings per train user:  4.715986795567083 
ratings per test user:  2.115656684744164


### Modeling

In [19]:
models = {}  # models['모델명'][0] => model 객체, models['모델명'][1] => model 파라미터

##### BIVAECF


In [20]:
LATENT_DIM = 50
ENCODER_DIMS = [100]
ACT_FUNC = "tanh"
LIKELIHOOD = "pois"
NUM_EPOCHS = 1000
BATCH_SIZE = 128
LEARNING_RATE = 0.001

In [21]:
params = {
    'k': LATENT_DIM,
    'encoder_structure':ENCODER_DIMS,
    'act_fn' : ACT_FUNC,
    'likelihood': LIKELIHOOD,
    'n_epochs': NUM_EPOCHS,
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE}

model = BiVAECF(**params, seed=SEED, verbose=VERBOSE)
models[model.name] = (model, params)
BiVAECF()

##### Embarrassingly Shallow Autoencoders for Sparse Data (EASE)

In [22]:
params = {
    'lamb': 40,
    'posB': True,
}

model = EASE(**params, seed=SEED, verbose=VERBOSE)
models[model.name] = (model, params)

##### LightGCN

In [36]:
params = {
    'num_layers': 3,
    'batch_size': 640,
    'learning_rate': 0.0007457359048142671,
    'lambda_reg' : 0.006838749269501347,
    'num_epochs': 50,
}

model = LightGCN(**params,seed=SEED, verbose=VERBOSE)
models[model.name] = (model, params)

##### WMF

In [25]:
params = {
    'k': 3,
    'max_iter': 390 ,
    'learning_rate': 0.011271662653605422,
    'lambda_u': 0,
    'lambda_v': 0,
    'batch_size': 512,
}

model = WMF(**params, seed=SEED, verbose=VERBOSE)
models[model.name] = (model, params)

##### NGCF

In [26]:
params = {
'emb_size': 112,
'learning_rate': 0.0045876134988093184,
 'lambda_reg': 0.008382902059879144
}

model = NGCF(**params, seed=SEED, verbose=VERBOSE)
models[model.name] = (model, params)

### Experiment

In [27]:
# 평가방법 설정
eval_method = BaseMethod.from_splits(
    train_data=np.array(train), 
    test_data=np.array(test), 
    exclude_unknowns=True,  # Unknown users and items will be ignored.
    verbose=True
)

#Random split
#ratio_split = RatioSplit(
#  data=df, test_size=0.2, exclude_unknowns=True, seed=SEED, verbose=VERBOSE
#)

#K-fold CV
#ratio_split = CrossValidation(
#  data=data, n_folds=5, exclude_unknowns=True, seed=SEED, verbose=VERBOSE
#)

rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 8482
Number of items = 6671
Number of ratings = 40001
Max rating = 1.0
Min rating = 1.0
Global mean = 1.0
---
Test data:
Number of users = 8452
Number of items = 6072
Number of ratings = 17868
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 8482
Total items = 6671


In [ ]:
%%time

# 평가척도 설정
metrics = [Recall(k=TOP_K), NDCG(k=TOP_K)]

# 실험 수행
cornac.Experiment(
    eval_method=eval_method,
    models=[m[0] for m in models.values()],
    metrics=metrics,
).run()

### Prediction

In [ ]:
# 전체 데이터 Cornac 데이터형식으로 변환
full_data = cornac.data.Dataset.from_uir(data.itertuples(index=False), seed=SEED)

# 모델 선택
model = LightGCN #BiVAECF, LightGCN, EASE, NGCF, WMF

# 전체 데이터로 다시 학습
model = model(**models[model().name][1], verbose=VERBOSE, seed=SEED)
model.fit(full_data)

In [ ]:
# All item에 대한 예측값 생성
all_pred = predict_ranking(
    model, data, 
    usercol=DEFAULT_USER_COL, itemcol=DEFAULT_ITEM_COL, 
    remove_seen=True
)

  0%|          | 0/8482 [00:00<?, ?it/s]

In [ ]:
# Top-K item 생성
top_k = (
    all_pred
    .groupby(DEFAULT_USER_COL)
    .apply(lambda x: x.nlargest(TOP_K, DEFAULT_PREDICTION_COL))
    .reset_index(drop=True)
    #.drop(DEFAULT_PREDICTION_COL, axis=1)
    .sort_values(by=DEFAULT_USER_COL)
)

# submission 화일 저장
t = pd.Timestamp.now()
fname = f"submit_{model.name}_{t.month:02}{t.day:02}{t.hour:02}{t.minute:02}.csv"
top_k.to_csv(fname, index=False)

<font color='tomato'><font color="#CC3D3D"><p>
# End